In [4]:
import zipfile
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

In [5]:
zip_file_path = r"C:\Users\yashs\python\images.zip"
extract_path = 'cifake_data'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [6]:
train_real_dir = os.path.join(extract_path, 'train', 'REAL')
train_fake_dir = os.path.join(extract_path, 'train', 'FAKE')
test_real_dir = os.path.join(extract_path, 'test', 'REAL')
test_fake_dir = os.path.join(extract_path, 'test', 'FAKE')

In [7]:
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(32, 32))
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = img / 255.0  
        images.append(img)
        labels.append(label)
    return images, labels

In [8]:
train_real_images, train_real_labels = load_images_from_folder(train_real_dir, 0)
train_fake_images, train_fake_labels = load_images_from_folder(train_fake_dir, 1)
test_real_images, test_real_labels = load_images_from_folder(test_real_dir, 0)
test_fake_images, test_fake_labels = load_images_from_folder(test_fake_dir, 1)

In [9]:
x_train = np.array(train_real_images + train_fake_images)
y_train = np.array(train_real_labels + train_fake_labels)
x_test = np.array(test_real_images + test_fake_images)
y_test = np.array(test_real_labels + test_fake_labels)

In [10]:
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [11]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(2, activation='sigmoid')
])

C:\Users\yashs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [13]:
history = model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 42s 25ms/step - accuracy: 0.7814 - loss: 0.4483 - val_accuracy: 0.8983 - val_loss: 0.2477
Epoch 2/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 36s 23ms/step - accuracy: 0.8985 - loss: 0.2497 - val_accuracy: 0.9129 - val_loss: 0.2120
Epoch 3/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 36s 23ms/step - accuracy: 0.9147 - loss: 0.2149 - val_accuracy: 0.9199 - val_loss: 0.1984
Epoch 4/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 36s 23ms/step - accuracy: 0.9262 - loss: 0.1876 - val_accuracy: 0.9258 - val_loss: 0.1813
Epoch 5/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 36s 23ms/step - accuracy: 0.9333 - loss: 0.1695 - val_accuracy: 0.9285 - val_loss: 0.1823
Epoch 6/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 36s 23ms/step - accuracy: 0.9418 - loss: 0.1514 - val_accuracy: 0.9305 - val_loss: 0.1717
Epoch 7/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 37s 24ms/step - accuracy: 0.9456 - loss: 0.1386 - val_accuracy: 0.9314 - val_loss: 0.1759
Epoch 8/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 36s 23ms/step - accuracy: 0.9508 -

In [14]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9415 - loss: 0.2291
Test accuracy: 0.9259499907493591


In [16]:
model.save('cifake_classifier.h5')
loaded_model = tf.keras.models.load_model('cifake_classifier.h5')

In [17]:
loaded_test_loss, loaded_test_acc = loaded_model.evaluate(x_test, y_test)
print(f'Loaded model test accuracy: {loaded_test_acc}')

625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9415 - loss: 0.2291
Loaded model test accuracy: 0.9259499907493591
